# Aligning WordNet to an Embedding Dictionary

In [1]:
from scipy.sparse import csr_matrix
from nltk.corpus import wordnet as wn
from tqdm import tqdm
from collections import Counter
import numpy as np
import networkx as nx
import pickle

First item of business, getting the embedding dictionary word list (so we know which words we want to map to synsets to begin with)

In [3]:
emb_tab_file = 'data/embeddings/en.emb.txt'
en_words = []
errors = []
with open(emb_tab_file, 'rb') as embs:
    for e in tqdm(embs):
        cols = e.decode("utf-8").split()
        if len(cols) < 3: continue
        try:
            float(cols[1])
        except ValueError:
            errors.append(' '.join(cols[:5]))
            continue
        en_words.append(cols[0])
print('\n'.join(errors))

200001it [00:15, 12617.39it/s]


10æ¥ä»¥å ã«çºé 0.039924000000000001 0.10034999999999999 0.042516999999999999
 the -0.098361000000000004 0.014168999999999999 0.039489999999999997
7æ¥ä»¥å ã«çºé 0.019684 0.059399 -0.0065339999999999999


In [4]:
len(en_words)

199997

In [5]:
en_words[:10]

['the', ',', '.', 'of', 'and', 'to', 'in', 'a', 'is', '"']

In [6]:
en_words[-5:]

['hameroff', 'gaultheria', 'margriet', 'katzen', '10-credit']

Now let's whip out WordNet and find alignments from synset to word(s).

Our first attempt is just using the WordNet interface's `synsets()` function, which is string-based. All synsets found for each word are mapped to it in `pairings`.

In [7]:
synsets = {}  # list of synsets, in order found by traversing embedding vocabl list
pairings = []  # list of 1-valued cells in matrix to be built
no_syns_found = 0  # counter for words with no synsets attached
for i,w in tqdm(enumerate(en_words)):
    syns = wn.synsets(w)
    if len(syns) == 0:
        no_syns_found += 1
        continue
    for sn in syns:
        if sn not in synsets:
            synsets[sn] = len(synsets)
        pairings.append((synsets[sn], i))
itosyns = {i:sn for sn,i in synsets.items()}
no_word = len([sn for sn in wn.all_synsets() if sn not in synsets])
print(f'found {len(pairings)} pairings for {len(synsets)} synsets and {len(en_words)} words.')
print(f'no wn records found for {no_syns_found} words.')
print(f'no embeddings found for {no_word} synsets.')

199997it [00:20, 9764.67it/s] 


found 186978 pairings for 71558 synsets and 199997 words.
no wn records found for 128945 words.
no embeddings found for 46101 synsets.


Sanity check. The word *in* might be a good example.

In [8]:
in_sns = [itosyns[j] for j in [p[0] for p in pairings if p[1] == en_words.index('in')]]
in_sns

[Synset('inch.n.01'),
 Synset('indium.n.01'),
 Synset('indiana.n.01'),
 Synset('in.s.01'),
 Synset('in.s.02'),
 Synset('in.s.03'),
 Synset('in.r.01')]

This is a problem. We don't really want these mappings for the clearly-prepositional word 'in'.
Maybe we should trim by lemma count? Case in point:

In [9]:
[[(l, l.count()) for l in sn.lemmas()] for sn in in_sns]

[[(Lemma('inch.n.01.inch'), 49), (Lemma('inch.n.01.in'), 3)],
 [(Lemma('indium.n.01.indium'), 0),
  (Lemma('indium.n.01.In'), 0),
  (Lemma('indium.n.01.atomic_number_49'), 0)],
 [(Lemma('indiana.n.01.Indiana'), 2),
  (Lemma('indiana.n.01.Hoosier_State'), 0),
  (Lemma('indiana.n.01.IN'), 0)],
 [(Lemma('in.s.01.in'), 0)],
 [(Lemma('in.s.02.in'), 0)],
 [(Lemma('in.s.03.in'), 0)],
 [(Lemma('in.r.01.in'), 19),
  (Lemma('in.r.01.inwards'), 0),
  (Lemma('in.r.01.inward'), 0)]]

OK. New strategy. Only link to synset if the corresponding lemma's count is nonzero, see where that gets us.

In [10]:
trimmed_synsets = {}  # list of synsets, in order found by traversing embedding vocabl list
idx_to_trimmed_synsets = {}  # inverted index
trimmed_pairings = []  # list of 1-valued cells in matrix to be built
no_trim_syns_found = 0  # counter for words with no synsets attached
for i,w in tqdm(enumerate(en_words)):
    lemmata = wn.lemmas(w)
    syns = [lm.synset() for lm in lemmata if lm.count() > 0]
    if len(syns) == 0:
        no_trim_syns_found += 1
        continue
    for sn in syns:
        if sn not in trimmed_synsets:
            j = len(trimmed_synsets)
            trimmed_synsets[sn] = j
            idx_to_trimmed_synsets[j] = sn
        trimmed_pairings.append((trimmed_synsets[sn], i))        
no_trim_word = len([sn for sn in wn.all_synsets() if sn not in trimmed_synsets])
print(f'found {len(trimmed_pairings)} pairings for {len(trimmed_synsets)} synsets and {len(en_words)} words.')
print(f'no wn records found for {no_trim_syns_found} words.')
print(f'no embeddings found for {no_trim_word} synsets.')

199997it [00:47, 4183.54it/s] 


found 29560 pairings for 23154 synsets and 199997 words.
no wn records found for 184018 words.
no embeddings found for 94505 synsets.


Hmm, that's some heavy trimming. We only have ~15k words with matches.
Although now it's only ~2 pairs per word, not ~3, I think we can add one more heuristic:
It's ok to pair a word with a synset if it's the only one corresponding to it, or if it's the only lemma in the synset.

In [11]:
v3_synsets = {}  # list of synsets, in order found by traversing embedding vocabl list
idx_to_v3_synsets = {}  # inverted index
v3_pairings = []  # list of 1-valued cells in matrix to be built
no_v3_syns_found = 0  # counter for words with no synsets attached
for i,w in tqdm(enumerate(en_words)):
    lemmata = wn.lemmas(w)
    if len(lemmata) == 1:
        syns = wn.synsets(w)
    else:
        syns = [lm.synset() for lm in lemmata if lm.count() > 0 or len(lm.synset().lemmas()) == 1]
    if len(syns) == 0:
        no_v3_syns_found += 1
        continue
    for sn in syns:
        if sn not in v3_synsets:
            j = len(v3_synsets)
            v3_synsets[sn] = j
            idx_to_v3_synsets[j] = sn
        v3_pairings.append((v3_synsets[sn], i))        
no_v3_word = len([sn for sn in wn.all_synsets() if sn not in v3_synsets])
print(f'found {len(v3_pairings)} pairings for {len(v3_synsets)} synsets and {len(en_words)} words.')
print(f'no wn records found for {no_v3_syns_found} words.')
print(f'no embeddings found for {no_v3_word} synsets.')

199997it [00:39, 5016.33it/s] 


found 80623 pairings for 60167 synsets and 199997 words.
no wn records found for 155851 words.
no embeddings found for 57492 synsets.


In [12]:
# super-sanity check
for j in range(len(v3_synsets)):
    assert j in idx_to_v3_synsets, j

Substantially better. 3 times as many vocab words found in WN, 2.6x as many pairings.

(Note: without the `or`-clause it was 2.5x vocab words, 2x pairings. This clause brings rows with a single entry, is this something we want? It's a concept that exists and might have a different word in the target language, although it might be too rare, something like an obscure meaning of *have*.)

TODO: what about **inflections** in word embedding vocab? Do we pair with the same synset(s)?
We probably shouldn't, since inflections don't carry across languages, but in this case what do we do with those words?

Another TODO is to retrain the embedding model to include ngram phrases that we know about in WN (i.e. preprocess the upstream corpus). Even if it is only done in English, presumably concepts in the target language might be more frequently single words.

## Alignment Graph Properties

Let's create a sparse graph object for simple querying and storage.

In [14]:
sorted_pairings = sorted(v3_pairings)
indices = []
indptr = []
x = -1
for idx,(j,i) in enumerate(sorted_pairings):
    if j != x:
        indptr.append(idx)
        x = j
    indices.append(i)
indptr.append(idx)
data = [1] * (idx+1)

pairings_graph = csr_matrix((data,indices,indptr), shape=(sorted_pairings[-1][0]+1, max([i for j,i in v3_pairings])+1))

Correctness check.

In [15]:
pairings_graph.get_shape()

(60167, 199994)

In [25]:
pairings_graph[56].nonzero()

(array([0, 0, 0, 0, 0, 0]),
 array([    40,     79,    114,    835,   3296, 126060]))

In [27]:
idx_to_v3_synsets[56].lemmas()

[Lemma('merely.r.01.merely'),
 Lemma('merely.r.01.simply'),
 Lemma('merely.r.01.just'),
 Lemma('merely.r.01.only'),
 Lemma('merely.r.01.but')]

In [28]:
en_words[40], en_words[114], en_words[3296], en_words[126060]

('but', 'just', 'merely', 'but')

Hrmph. That second *but* has some unicode thing `U+0085 NEXT LINE CHARACTER` attached to it that was removed when processing the embeddings file.

TODO get rid of these, there's 77 of them, 55 of whom at word ends, who knows what other characters may be lurking :-/

Now let's talk degree distributions.

In [29]:
# words per synset
x_degs = pairings_graph.sum(axis=1).flatten().tolist()[0]
xdeg_counts = Counter(x_degs)
xdeg_counts

Counter({0: 1,
         1: 47524,
         2: 8465,
         3: 2404,
         4: 899,
         5: 415,
         6: 215,
         7: 115,
         8: 62,
         9: 32,
         10: 15,
         11: 8,
         12: 4,
         13: 1,
         14: 3,
         15: 3,
         20: 1})

TODO: check who that '0' is and remove.

In [30]:
# synsets per word
y_degs = pairings_graph.sum(axis=0).flatten().tolist()[0]
ydeg_counts = Counter(y_degs)
ydeg_counts

Counter({0: 155849,
         1: 30499,
         2: 6453,
         3: 2908,
         4: 1430,
         5: 938,
         6: 514,
         7: 401,
         8: 264,
         9: 156,
         10: 121,
         11: 84,
         12: 84,
         13: 52,
         14: 43,
         15: 38,
         16: 29,
         17: 13,
         18: 21,
         19: 18,
         20: 9,
         21: 6,
         22: 7,
         23: 8,
         24: 2,
         25: 2,
         26: 6,
         27: 2,
         28: 3,
         29: 7,
         30: 2,
         31: 4,
         32: 1,
         33: 1,
         34: 3,
         35: 1,
         36: 1,
         37: 4,
         38: 1,
         39: 1,
         43: 2,
         44: 2,
         45: 1,
         56: 1,
         60: 2})

This actually looks pretty sane.

We'll write the matrix and index dictionaries to files for use in the algorithm. Synsets don't have to be qualified by name for the currently-designed algorithm, but we'll keep the name-mapping anyway, by list of their string names.

In [31]:
# write to files
if False:  # change when actually writing
    synset_list = [sn.name() for _,sn in sorted(idx_to_v3_synsets.items(), key = lambda x: x[0])]
    with open('data/v3a_synsets.txt', 'w') as synset_list_file:
        for sn in synset_list:
            synset_list_file.write(f'{sn}\n')

    with open('data/v3a_wordlist.txt', 'w', encoding='utf8') as wordlist_file:
        for w in en_words:
            wordlist_file.write(f'{w}\n')

    with open('data/v3a_pairings.pkl', 'wb') as graph_file:
        pickle.dump(pairings_graph, graph_file)

# Inducing WN Structure on Limited Dataset

Let's continue with the structural properties of our induced WordNet (for phase II of the project).

I'll copy over some things from [the M3GM notebook](https://github.com/yuvalpinter/m3gm/blob/master/wn_exploration/wordnet_analysis.ipynb).

In [32]:
def nonempty_rels_count(rel, dset=v3_synsets) -> int:
    """
    Computes out degree
    """
    return len([s for s in dset if len([t for t in rel(s) if t in dset]) > 0])

print(f'Total synsets: {len(v3_synsets):,}')
rel_names = ['hypernyms', 'hyponyms', 'entailments', 'attributes', 'causes', 'member meronyms', \
             'substance meronyms', 'part meronyms', 'member holonyms', 'substance holonyms', 'part holonyms', 'similar tos']
rel_funcs = [lambda s: s.hypernyms(), lambda s: s.hyponyms(), lambda s: s.entailments(), lambda s: s.attributes(),\
                      lambda s: s.causes(), lambda s: s.member_meronyms(), lambda s: s.substance_meronyms(),\
                      lambda s: s.part_meronyms(), lambda s: s.member_holonyms(), lambda s: s.substance_holonyms(), \
                      lambda s: s.part_holonyms(), lambda s: s.similar_tos()]

for name, rel in zip(rel_names, rel_funcs):
    print(f'{nonempty_rels_count(rel):5d} have {name}')


Total synsets: 60,167
32494 have hypernyms
 9411 have hyponyms
  311 have entailments
  707 have attributes
  180 have causes
  639 have member meronyms
  200 have substance meronyms
 1342 have part meronyms
 1013 have member holonyms
  175 have substance holonyms
 2936 have part holonyms
 8600 have similar tos


In [33]:
def graphify(relation) -> nx.DiGraph:
    """
    Create a networkx directed graph out of a WordNet synset relation type
    """
    relg = nx.DiGraph()
    relg.add_nodes_from(v3_synsets)
    for s in v3_synsets:
        targets = [t for t in relation(s) if t in v3_synsets]
        relg.add_edges_from(zip([s] * len(targets), targets))
    return relg

In [34]:
hypg = graphify(lambda s: s.hyponyms())
hypg.remove_edge(wn.synset('inhibit.v.04'), wn.synset('restrain.v.01'))
nx.dag.is_directed_acyclic_graph(hypg)

True

In [35]:
in_out_degs = {n:(i,o) for ((n,i),(n,o)) in zip(hypg.in_degree(), hypg.out_degree())}
roots = [n for n,(i,o) in in_out_degs.items() if i == 0 and o > 0]

print(f'Number of hyponym graph roots is {len(roots)}')
subtrees = {r:len(nx.dfs_tree(hypg, r)) for r in roots}
print('Roots with largest subgraphs:')
for x in sorted(subtrees.items(), key=lambda x:-x[1])[:10]:
    print(f'{x[0].name()} : {x[1]:,}')

Number of hyponym graph roots is 1963
Roots with largest subgraphs:
organism.n.01 : 2,771
causal_agent.n.01 : 2,764
artifact.n.01 : 2,721
event.n.01 : 2,346
attribute.n.02 : 2,004
matter.n.03 : 1,085
change.v.01 : 1,059
relation.n.01 : 946
cognition.n.01 : 871
move.v.02 : 817


This is more roots than we're used to. Most oddly of all, where's `entity.n.01`? (TODO)

Other than that, the big ones seem to be there.

The following are snippets from the M3GM network with results copied over as line comments.

In [36]:
act_root = wn.synset('act.v.01')
move_root = wn.synset('move.v.02')
act_st = nx.dfs_tree(hypg, act_root)
move_st = nx.dfs_tree(hypg, move_root)
print(f'Act has {len(act_st):,} hyponyms, Move has {len(move_st):,}') # verification
print(f'They have {len([s for s in act_st if s in move_st])} hyponyms in common') # intersection (not necessarily 0, since this is a DAG)

Act has 771 hyponyms, Move has 817
They have 0 hyponyms in common


In [37]:
def sc_frac(tree):
    """
    Returns the fraction of nodes in a tree that only have one child
    """
    return len([d for _,d in tree.out_degree() if d == 1]) / len(tree)

print(f'Single-child fraction for Act is {sc_frac(act_st):.5f}')  # around 0.11 in full graph
print(f'Single-child fraction for Move is {sc_frac(move_st):.5f}')  # around 0.11 in full graph

Single-child fraction for Act is 0.11673
Single-child fraction for Move is 0.10526


In [38]:
def all_depths(tree, root):
    return [nx.shortest_path_length(tree, root, n) for n in tree]

def med_depth(tree, root):
    return int(np.median(all_depths(tree, root)))

print(f'Median depth for Act is {med_depth(act_st, act_root)}')  # 5 in full graph
print(f'Median depth for Move is {med_depth(move_st, move_root)}')  # 3 in full graph

Median depth for Act is 4
Median depth for Move is 3


In [39]:
def tree_width(tree, root):
    return Counter(all_depths(tree, root)).most_common(1)[0][1]

print(f'Width for Act tree is {tree_width(act_st, act_root)}')  # 226 in full graph
print(f'Width for Move tree is {tree_width(move_st, move_root)}')  # 311 in full graph

Width for Act tree is 174
Width for Move tree is 252


In [40]:
def pos_connected(e, directed=True) -> tuple:
    """
    A function telling us which parts-of-speech are participating in an edge
    :param e: an nx edge
    :param directed: True iff we care about which node comes first (not so for polysemy)
    """
    if not directed:
        return tuple(sorted([e[0].pos(), e[1].pos()]))
    return (e[0].pos(), e[1].pos())

def pos_connections(graph: nx.Graph, directed=True) -> Counter:
    """
    Counts part-of-speech relations in a graph.
    """
    return Counter([pos_connected(e, directed) for e in graph.edges()])

def pos_connection_viz(graph):
    print('\n'.join([f'{p1} -> {p2}: {ct:,} times' for (p1,p2),ct in pos_connections(graph).most_common()]))

print('Hyponym connections between parts of speech:')
pos_connection_viz(hypg)  # in original graph: 75,850 n -> n, 13,238 v -> v

Hyponym connections between parts of speech:
n -> n: 23,306 times
v -> v: 9,541 times


Meaning that the verb synsets were better-preserved, which makes sense due to the longer-tailedness of the noun concept domain.

All-in-all, this appears to be a reasonable graph that can be useful as an embedding projection signal, and may be used in an ERGM estimation protocol.